
# Классификация изображений

### Основная идея этого решения: взять предобученую на ImageNet сеть Xception и дообучить под нашу задачу. 
По ходу решения мы будем давать вам рекомендации, которые помогут улучшить качество модели. 


Удачи и Поехали!

v3

## Выводы

Сразу со сложностей - столкнуалсь со сложностями, так как превышала лимит в 30 часов в неделю. Осталось для меня не выяснено как в датасете 1-model, куда я вручную переносила файлы моделей от итерации к итерации (ограничение на 8 или 9 часов работы сессии GPU) - как в датасете ходить от версии к версии.

**По работе:** 

были эксперименты, где модель начиналась с базовой Efficient Net B5 (efnb5) и обучалась целиком - где-то на 80 эпoхах обучение закончилось по earlystop (patience=20) и точность модели была 81% - по времени ушло почти 8 часов

были эксперименты, с efnb5 и xception - где базовые модели замораживались на обучении - efn давало точность выше на процентов 7-8 - тут было мало эпох, порядка 20. Было принято решение дальше экспереминтировать только с efn

efn как базовая модель замораживалась, и потом было 3 этапа fine-tuning - доли размораживающихся слоев выбирались исходя из мысли - постепенно увеличивать количество обучаемых параметров модели. Для EFN подобрала сначала разморозить 1/8 часть на обучение, потом 1/4, потом все слои. 
Понравлся такой вариант тем, что он по времени показадся более эффективным. Возможно так "сильно" уменьшать LR от итерации fine-tuninga к слпедующей не стоит, но времени GPU пока нет, чтоб это проверить. Без GPU беспощадно долго считать (на 5 эпох уходило порядка 9-10 часов О_о)

Из досадного - влетала в ходе экспериментов на превышение объемов памяти, уменьшала размер batch-ей, но время GPU было уже не вернуть, так как эксперименты ставились на ночь.
Любопытно - нет ли каких-то вариантов предвидеть такие вещи, как-то просчитать - не понимаю от каких параметров и хоть примерно как (линейно, квадратично или еще как ) зависит объем памяти, клоторый нужен будет во время вычислений (в своих "привычных" задачах из криптографии пользуюсь таким предвариетльным анализом - а тут не знаю как)

In [ ]:
print('ALOHA! 2')

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, \
                                        EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
import tensorflow.keras.models as Model
from tensorflow.keras.applications.xception import Xception
import tensorflow.keras.layers as Layer


from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

# Основные настройки

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.


EPOCHS               = 75  # эпох на обучение
BATCH_SIZE           = 16 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/'
# PATH = "../working/car/" # рабочая директория
PIC_PATH = "/kaggle/temp/" #  директория для картинок, должна очиститься сама
WORK_PATH = '../working/car/'
    


In [ ]:
# !mkdir /kaggle/temp
!ls -l /kaggle/input/

In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(WORK_PATH,exist_ok=False)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо, классов - 10 шт

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
        z.extractall(PIC_PATH)
        
print(os.listdir(PIC_PATH))

In [ ]:
!ls -l /kaggle/temp

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PIC_PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PIC_PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

### Уже догадываетесь, что означают классы?
Это будут модели машин

# Подготовка данных

### Аугментация данных

In [ ]:
# Вы помните, что аугментация данных важна, когда мы работаем с небольшим датасетом.
# Это как раз наш случай.
# Чтобы лучше понять работу параметров, попробуйте их изменить.
# К какому результату это приведет?
# Официальная документация: https://keras.io/preprocessing/image/

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 5,
    zoom_range=[0.75,1.25],
    brightness_range=[0.5, 1.5],
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=VAL_SPLIT, # set validation split
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

#Рекомендация Подключите более продвинутые библиотеки аугментации изображений
# (например: albumentations или imgaug, для них есть специальные "обертки" под Keras, 
# например: https://github.com/mjkvaak/ImageDataAugmentor)

### Генерация данных

In [ ]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PIC_PATH+'train/',      # директория где расположены папки с картинками 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    PIC_PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PIC_PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

# Обратите внимание, что для сабмита мы используем другой источник
# test_datagen.flow_from_dataframe. Как вы думаете, почему?
# Потому что нам нужно дать предсказание с привязкой
# к конкретному Id картинки из dataframe.

Посмотрим на примеры картинок после аугментации

In [ ]:
from skimage import io

def imshow(image_RGB):
  io.imshow(image_RGB)
  io.show()

x,y = train_generator.next()
print('Пример картинок из train_generator')
plt.figure(figsize=(12,8))

for i in range(0,6):
    image = x[i]
    plt.subplot(3, 3, i+1)
    plt.imshow(image)
    #plt.title('Class: '+str(y[i]))
    #plt.axis('off')
plt.show()

# Построение модели

### Загружаем предобученную сеть  EfficientNetB5  (ver1 Xception):

In [ ]:
!pip install -q efficientnet

In [ ]:
import efficientnet.keras as efn 

In [ ]:
# base_model = Xception(weights='imagenet',
#                       include_top=False,
#                       input_shape = input_shape)

base_model = efn.EfficientNetB5(weights='imagenet', 
                                include_top=False, 
                                input_shape = input_shape)

In [ ]:
base_model.summary()


Заморозим первый слой базовой модели Xception/Efnb5

In [ ]:
base_model.trainable = False

In [ ]:
# Устанавливаем новую "голову" (head)

model=Model.Sequential()
model.add(base_model)
model.add(Layer.GlobalAveragePooling2D())
model.add(Layer.Dense(256, 
                      activation='relu'
                     ))
model.add(Layer.BatchNormalization())
model.add(Layer.Dropout(0.25))
model.add(Layer.Dense(CLASS_NUM, activation='softmax'))

model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [ ]:
model.summary()


## Обучение модели

Добавим ModelCheckpoint, чтоб сохранять прогресс обучения модели, и можно было потом подгрузить и дообучить модель.

EarlyStopping - отвечает за остановку обучения модели, если метрика не улучшается на протяжении какого-то количества эпох (patience)

ReduceLROnPlateau - при неулучшении метрики (тут val_loss) будем изменять learning-rate на  factor

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy',
                             verbose=1, save_best_only=True, mode='max')

earlystop = EarlyStopping(monitor='val_accuracy', 
                          patience=10, 
                          restore_best_weights=True,
                          verbose=1)

reduceLR = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=4,
                              min_lr=0.0000001,
                              verbose=1)

callbacks_list = [checkpoint, earlystop, reduceLR]

# Рекомендация 1. Добавьте другие функции из https://keras.io/callbacks/
# Рекомендация 2. Используйте разные техники управления Learning Rate
# https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng)
# http://teleported.in/posts/cyclic-learning-rate/ (eng)

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

Обучаем:

In [ ]:
# history = model.fit_generator(
#         train_generator,
#         steps_per_epoch = len(train_generator),
#         validation_data = test_generator, 
#         validation_steps = len(test_generator),
#         epochs = EPOCHS,
#         callbacks = callbacks_list
# )

# # Рекомендация: попробуйте применить transfer learning с fine-tuning

In [ ]:
# # сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
# model.save('/kaggle/working/model_last.hdf5')
# model.load_weights('/kaggle/working/best_model.hdf5')
# model.save('/kaggle/working/best_model_before_ft.hdf5')



## Шаг тюнинга

тут загрузим модель, которую получили на первом шаге, в ней первый слой был заморожен для обучения (был эксперимент, где EfficientnetB5 со 100 эпохами имел точность 81% точности на тесте - **todo - разобраться как откатываться к разным версиям данных в 1-model, где сохранялись файлы best_model**).

(Как происхоило это все в kaggle - модель ставилась на Run&Save в фоновом режиме. Когда модель завершала работу -то в Output сохранялся файл best_model.hdf5. Его было добавлено как New Dataset в рабочую среду, и теперь он есть в Input (1-model). При изменении начальной модеди (первого шага - получения best_model) из Output надо опять "забирать" этот "датасет" кнопкой например New Version. И в рабочей среде обновляется файлик. Сейчас будем его доставать и загружать)

И да начнется fine-tuning

## Загружаем модель

In [ ]:
# !ls -l /kaggle/input/1-model/

In [ ]:
# # если модель уже есть - то скопируем ее из 1-model в рабочую папку
# !cp /kaggle/input/1-model/best_model.hdf5 /kaggle/working/best_model.hdf5

In [ ]:
# !pip install -q efficientnet

# import efficientnet.keras as efn 

In [ ]:
model = Model.load_model('/kaggle/working/best_model_before_ft.hdf5')
model.save('/kaggle/working/best_model.hdf5')

In [ ]:
model.summary()

в слое efn размлорозим часть параметров для обучения - последнюю 1/8 слоев - это добавит порядка (15 млн параметров в обучение для efnb5 / 7 млн параметров для xception).

In [ ]:
# bm = model.get_layer(name='xception')
bm = model.get_layer(name='efficientnet-b5')



fine_tune_at = len(bm.layers)//8 * 7

bm.trainable=True
# Freeze all the layers before the `fine_tune_at` layer
for layer in bm.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
fine_tune_at

In [ ]:
model.summary()

На весь fine_tuning возьмем меньше эпох

In [ ]:
EPOCHS               = 25  # эпох на обучение

In [ ]:
LR=0.0001
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
model.save('/kaggle/working/model_ft_1.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

В Итоге точность нашей модели составила __%. 



Посмотрим графики обучения:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

## 2 этап fine-tuning'a

Продолжим fine-tuning

In [ ]:
# bm = model.get_layer(name='xception')
# fine_tune_at = len(bm.layers)//2
bm = model.get_layer(name='efficientnet-b5')
fine_tune_at = len(bm.layers)//4 *3

bm.trainable=True
# Freeze all the layers before the `fine_tune_at` layer
for layer in bm.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
model.summary()

разморозили четверть слоев базовой модели efn-b5, это дало порядка (20 млн параметров для efnb5 / 15 млн для xception - выбрали другой шаг fine_tune_at - чтоб добаить больше параметров) для обучения

уменьшаем LR

In [ ]:
LR=0.00001
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
model.save('/kaggle/working/model_ft_2.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

В Итоге точность нашей модели составила __%. 

  

Посмотрим графики обучения:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

## 3 этап

разморозим все слои

In [ ]:
# bm = model.get_layer(name='xception')
bm = model.get_layer(name='efficientnet-b5')
fine_tune_at = 0

bm.trainable=True
# Freeze all the layers before the `fine_tune_at` layer
for layer in bm.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
model.summary()

In [ ]:
LR=0.000005
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=LR), 
              metrics=["accuracy"])

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
model.save('/kaggle/working/model_ft_3.hdf5')
try:
    model.load_weights('best_model.hdf5')
except:
    model = Model.load_model('/kaggle/working/best_model.hdf5')
model.save('/kaggle/working/model_final.hdf5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

В Итоге точность нашей модели составила __%. 



Посмотрим графики обучения:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# Предсказание на тестовых данных

In [ ]:
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

# Рекомендация: попробуйте добавить Test Time Augmentation (TTA)
# https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [ ]:
submission.head()

In [ ]:
# Clean PATH
# import shutil
# shutil.rmtree(PATH)

## Интересно, к какому классу модель отнесет вот эти автомобили?

# Что можно сделать, чтобы улучшить результат:

* Примените transfer learning с fine-tuning
* Настройте LR, optimizer, loss
* Подберите другие переменные (размер картинки, батч и т.д.)
* Попробуйте и другие архитектуры сетей (а не только Xception) или их ансамбли. Примеры SOTA на ImageNet  
* 
* Добавьте Batch Normalization и поэкспериментируйте с архитектурой “головы”
* Примените другие функции callback Keras https://keras.io/callbacks/ 
* Добавьте TTA (Test Time Augmentation)
* Дополнительно*: Используйте разные техники управления Learning Rate (https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng) http://teleported.in/posts/cyclic-learning-rate/ (eng))
* Дополнительно*: Добавьте более продвинутые библиотеки аугментации изображений (например, Albumentations )

### Удачи в соревновании!